In [ ]:
import sys
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# --- DATASET 1: E-COMMERCE (Fraud_Data) ---
ecommerce_df = pd.read_csv('../data/processed/processed_fraud_data.csv')

In [5]:
# Select engineered features + original numeric ones
e_features = ['purchase_value', 'age', 'time_since_signup', 'hour_of_day', 
              'day_of_week', 'device_id_count', 'ip_address_count']
X_e = ecommerce_df[e_features]
y_e = ecommerce_df['class']

X_train_e, X_test_e, y_train_e, y_test_e = train_test_split(
    X_e, y_e, test_size=0.2, random_state=42, stratify=y_e
)

In [6]:
# --- DATASET 2: BANK CREDIT (creditcard.csv) ---
credit_df = pd.read_csv('../data/raw/creditcard.csv')

In [8]:
# Pre-processing: Scale 'Time' and 'Amount' (V1-V28 are already scaled/PCA'd)
scaler = StandardScaler()
credit_df['Time'] = scaler.fit_transform(credit_df[['Time']])
credit_df['Amount'] = scaler.fit_transform(credit_df[['Amount']])

X_c = credit_df.drop('Class', axis=1)
y_c = credit_df['Class']

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_c, y_c, test_size=0.2, random_state=42, stratify=y_c
)